<a href="https://colab.research.google.com/github/tiagosardi/optimizationMethod/blob/main/branchAndBound.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Branch and Bound

Branch and bound serve para qualquer problema com variáveis inteiras (podendo ser linear ou não)

Fará uma avaliação de todas as possibilidades de melhor vizinho. Ao perceber que o custo de uma vizinha já excede o custo de algum vizinho já avaliado, o algoritmo desconsidera este caminho, sem precisar continuar o percurso.

Bound -> Limite (melhor custo)

f($\bar{x}$) = c(x) + h (custo + função heuristica)

a função h vai estimar o melhor caso (pode inicialmente ser 0)

Em programação linear, a melhor forma de calcular o bound é resolver o simplex relaxado, pois não tem a restrição integralidade (que força q as variaveis sejam inteiras). Então podemos fixar as variáveis e resolvemos o simplex que vai encontrar valores quebrados, mas aproximados.

O branch and bound é parecido com o brute force com permutação, só que é mais aprimorado.

In [ ]:
!pip install mip


# Problem Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from itertools import product
from sys import stdout as out
from mip import Model, xsum, minimize, BINARY, INTEGER

In [ ]:
n = 15
points = np.random.rand(n,2)

for p in points:
  plt.plot(p[0],p[1],'bo')

In [ ]:
dist_matrix = [[np.linalg.norm(np.array(p1)-np.array(p2)) for p1 in points] for p2 in points]


In [ ]:
n = len(points)
V = set(range(len(points)))

#Python MIP

In [ ]:
model = Model()


In [ ]:
x = [[model.add_var(var_type=BINARY) for j in V] for i in V]
y = [model.add_var(var_type=INTEGER,lb=1,ub=n-1) for i in V]

In [ ]:
model.objective = minimize(xsum(dist_matrix[i][j]*x[i][j] for i in V for j in V))


In [ ]:
for i in V:
  model += xsum(x[i][j] for j in V - {i}) == 1 

for j in V:
  model += xsum(x[i][j] for i in V - {j}) == 1 

for (i, j) in product(V - {0}, V - {0}):
  if i!=j:
    model += y[i] - y[j] + (n+1)*x[i][j] <= n

In [ ]:
model.optimize()
model.objective_value

In [ ]:
for row in x:
  vals = [e.x for e in row]
  print(vals)

In [ ]:

for i in range(len(x)):
  for j in range(len(x)):
    if x[i][j].x == 1:
      plt.plot([points[i][0],points[j][0]],[points[i][1],points[j][1]],'bo-')

In [ ]:

# [0,1,2,3,4]
def tsp_perm(x):
  sum = 0
  for i in range(len(x)):
    sum += np.linalg.norm(points[x[i]] - points[x[(i+1)%len(x)]])

  return sum

In [ ]:
def h(cities_to_visit): #recebe o numero de cidades q tá faltando
  return cities_to_visit*np.min(dist_matrix) #multiplica pela menor distancia possivel (constada na matriz)

In [ ]:

def branch_and_bound_perm(fobj,n,partial_solution=[],best_solution=[],best_val=np.inf,print_sol=False, h=lambda x:0,count=0):

  if len(partial_solution) == n: #se a solução estiver completa (condição de parada)
    
    fx = fobj(partial_solution) #ele avalia a solução
    
    

    if fx <= best_val: #se temos uma nova melhor solução
      if print_sol:
        print('{} : {}'.format(partial_solution,fx))
        best_solution = partial_solution #substitui a melhor solução
        best_val = fx # substitui o melhor valor  

    return best_solution,best_val,count
  
  else: # se a solução ainda não estiver completa
    for e in set(np.arange(n)).difference(set(partial_solution)): #para cada elemento disponível no domínio (cidade faltando add)
      count+= 1
      #inicio do trecho que diferencia o branch and bound do brute force
      cost = fobj(partial_solution + [e]) #a função obj vai pegar o vetor de cidades e calcula as distancias entre as cidades
      if cost+h(n-len(partial_solution)) <= best_val: #best_val é meu bound. Repare que ele vai ignorar caminhos que já estouraram o custo quando analisado com o melhor valor
        # h(n-len(partial_solution) pega as cidades q estão faltando
        #chama recursivamente, adicionando a cidade nova na solução parcial que temos atualmente
        best_solution,best_val,count = branch_and_bound_perm(fobj,n,partial_solution + [e], 
                  best_solution,
                  best_val,
                  print_sol,
                  h,count)
    
    return best_solution,best_val, count

In [ ]:

# a função objetivo será o tsp_perm
best_solution,best_val, count = branch_and_bound_perm(tsp_perm,n, h=h)

In [ ]:
best_solution,best_val, count 